In [4]:
from numpy import *
from scipy.sparse import *

In [5]:
trackArtist=load("track_artist.npy")
trackArtist

array([[2972914,     144],
       [2750239,     246],
       [1550729,     144],
       ..., 
       [2584455,   27938],
       [2299706,  373892],
       [2739985,  567363]], dtype=int32)

In [6]:
trackArtist.shape

(100000, 2)

In [6]:
playlistTracks=load_npz("all_playlists_with_tracks.npz")
playlistTracks

<45649x200 sparse matrix of type '<class 'numpy.int64'>'
	with 1086171 stored elements in Compressed Sparse Column format>

In [20]:
playlistTracks[:,0].data

array([    7569,     7614,     7641, ..., 11765405, 11766293, 11766362])

In [14]:
playlistTracks.shape

(45649, 200)

In [41]:
playlistTracks[:,1:][1].data.shape

(11,)

In [84]:
myList=[x.data.tolist() for x in playlistTracks[:,1:]]

In [85]:
backupList=myList.copy()

In [87]:
len(myList)

45649

In [86]:
#myList[0]#queste erano le tracce

[2634448, 2693660, 2227105, 2654612, 162463, 1195736, 2861386, 795606, 421750]

In [88]:
tIDs=trackArtist[:,0]
tIDs

array([2972914, 2750239, 1550729, ..., 2584455, 2299706, 2739985], dtype=int32)

In [89]:
artistIDs=trackArtist[:,1]
artistIDs

array([   144,    246,    144, ...,  27938, 373892, 567363], dtype=int32)

In [97]:
for i,tracks in enumerate(myList):
    if(i%10000==0):
        print(tracks)
    myList[i]=list(map(lambda x: artistIDs[where(tIDs==x)[0][0]], tracks ))

[2634448, 2693660, 2227105, 2654612, 162463, 1195736, 2861386, 795606, 421750]
[3435317]
[1891218, 234607, 1966460, 3830948, 1838617, 2988391, 356765, 3701295, 656735, 2712377, 3674375]
[20355, 1232476]
[3382133, 2838946, 1165920, 3117823, 1079342]


In [106]:
#myList[0]#tracce trasformate in artisti

[20683, 281045, 22096, 281045, 319024, 319024, 449220, 54765, 53362]

In [109]:
paddedList=list(map(lambda x: x+[0]*(199-len(x)) , myList))

In [114]:
all_playlists_with_artist=csc_matrix(paddedList)

In [116]:
all_playlists_with_artist

<45649x199 sparse matrix of type '<class 'numpy.int64'>'
	with 1040522 stored elements in Compressed Sparse Column format>

In [117]:
save_npz("all_playlists_with_artist.npz",all_playlists_with_artist)

In [7]:
all_playlists_with_artist=load_npz("all_playlists_with_artist.npz")

In [4]:
all_playlists_with_artist

<45649x199 sparse matrix of type '<class 'numpy.int64'>'
	with 1040522 stored elements in Compressed Sparse Column format>

In [8]:
trackArtist=load("track_artist.npy")
trackArtist,trackArtist.shape

(array([[2972914,     144],
        [2750239,     246],
        [1550729,     144],
        ..., 
        [2584455,   27938],
        [2299706,  373892],
        [2739985,  567363]], dtype=int32), (100000, 2))

In [132]:
maxArtistID=max(trackArtist[:,1])
maxSize=maxArtistID+1
maxArtistID

570588

In [114]:
firstRow=csc_matrix(bincount(all_playlists_with_artist[0].data,minlength=maxSize))
for i in range(1,45649):
    firstRow=vstack((firstRow,csc_matrix(bincount(all_playlists_with_artist[i].data,minlength=maxSize))))

In [112]:
firstRow=bincount(all_playlists_with_artist[0].data,minlength=maxSize)
firstRow=csc_matrix(firstRow)

In [113]:
vstack((firstRow,firstRow))

<2x570589 sparse matrix of type '<class 'numpy.int64'>'
	with 14 stored elements in COOrdinate format>

In [134]:
fullPlaylistArtist=csc_matrix(firstRow)
fullPlaylistArtist

<45649x570589 sparse matrix of type '<class 'numpy.int64'>'
	with 676931 stored elements in Compressed Sparse Column format>

In [165]:
onlyArtists=trackArtist[:,1]
firstTrack=csc_matrix(bincount(array([onlyArtists[0]]),minlength=maxSize))
for i in range(1,100000):
    firstTrack=vstack((firstTrack,csc_matrix(bincount(array([onlyArtists[i]]),minlength=maxSize))))

In [166]:
fullTrackArtist=csc_matrix(firstTrack)
fullTrackArtist

<100000x570589 sparse matrix of type '<class 'numpy.int64'>'
	with 100000 stored elements in Compressed Sparse Column format>

In [203]:
for i in range(100000):
    if(fullTrackArtist.getcol(i).size!=0):#same with fullPlaylist
        print(i)
        break

78


In [201]:
fullTrackArtist.getcol(78)

<100000x1 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Column format>

In [202]:
fullPlaylistArtist.getcol(78)

<45649x1 sparse matrix of type '<class 'numpy.int64'>'
	with 73 stored elements in Compressed Sparse Column format>

In [205]:
unique(onlyArtists).size

17536

In [208]:
firstArtistReduced=fullTrackArtist.getcol(78)
firstPlaylistReduced=fullPlaylistArtist.getcol(78)
for i in range(79,570589):
    thisTrackCol=fullTrackArtist.getcol(i)
    if(thisTrackCol.size!=0):
        firstArtistReduced=hstack((firstArtistReduced,thisTrackCol))
        firstPlaylistReduced=hstack((firstPlaylistReduced,fullPlaylistArtist.getcol(i)))    

In [215]:
allPlaylistArtistReduced=firstPlaylistReduced
allPlaylistArtistReduced

<45649x17536 sparse matrix of type '<class 'numpy.int64'>'
	with 676931 stored elements in Compressed Sparse Column format>

In [214]:
allTracksArtistReduced=firstArtistReduced
allTracksArtistReduced

<100000x17536 sparse matrix of type '<class 'numpy.int64'>'
	with 100000 stored elements in Compressed Sparse Column format>

In [213]:
save_npz("allPlaylistArtistReduced.npz",allPlaylistArtistReduced)
save_npz("allTracksArtistReduced.npz",allTracksArtistReduced)

In [21]:
allPlaylistArtistReduced=load_npz("allPlaylistArtistReduced.npz")
allTracksArtistReduced=load_npz("allTracksArtistReduced.npz")
allPlaylistArtistReduced,allTracksArtistReduced

(<45649x17536 sparse matrix of type '<class 'numpy.int64'>'
 	with 676931 stored elements in Compressed Sparse Column format>,
 <100000x17536 sparse matrix of type '<class 'numpy.int64'>'
 	with 100000 stored elements in Compressed Sparse Column format>)

In [14]:
allTrackIDs=trackArtist[:,0]
allTrackIDs,allTrackIDs.size#order of tracks_final.csv

(array([2972914, 2750239, 1550729, ..., 2584455, 2299706, 2739985], dtype=int32),
 100000)

In [15]:
targetTrackIDs=genfromtxt("target_tracks.csv",skip_header=1,dtype=int32)
targetTrackIDs

array([1316175, 3885714, 3091270, ..., 2739213, 2228646, 2265463], dtype=int32)

In [32]:
allPlaylistIDs=load_npz("all_playlists_with_tracks.npz")[:,0].data
allPlaylistIDs,playlistIDs.size

(array([    7569,     7614,     7641, ..., 11765405, 11766293, 11766362]),
 45649)

In [31]:
targetPlaylistIDs=genfromtxt("target_playlists.csv",skip_header=1,dtype=int32)
targetPlaylistIDs

array([10024884, 10624787,  4891851, ..., 11369546,  7939535,   297021], dtype=int32)

In [33]:
where(allPlaylistIDs==targetPlaylistIDs[0])[0][0]

36739

In [34]:
allPlaylistArtistReduced.getrow(36739)

<1x17536 sparse matrix of type '<class 'numpy.int64'>'
	with 43 stored elements in Compressed Sparse Row format>

In [36]:
firstTargetPlaylist=allPlaylistArtistReduced.getrow(36739)#prima per iniziare
for targetPlID in targetPlaylistIDs[1:]:#dalla seconda in poi si aggiungono
    toStack=allPlaylistArtistReduced.getrow(where(allPlaylistIDs==targetPlID)[0][0])
    firstTargetPlaylist=vstack((firstTargetPlaylist,toStack))

In [37]:
firstTargetPlaylist

<10000x17536 sparse matrix of type '<class 'numpy.int64'>'
	with 231470 stored elements in Compressed Sparse Row format>

In [39]:
save_npz("targetPlaylistArtistReduced.npz",firstTargetPlaylist)

In [45]:
onlyArtists=trackArtist[:,1]

In [60]:
save("uniqueArtists_NeededToIndexThe_ArtistReducedMatrices.npy",unique(onlyArtists))

#se la artist reduced contiene un valore in posizione N,
#bisogna prendere da qui l'artista in posizione N per sapere chi è


In [44]:
#some verification
targetPlaylistArtistReduced=load_npz("targetPlaylistArtistReduced.npz")
targetPlaylistArtistReduced[1].nonzero()[1]#playlist 10624787
#contains tracks 2090319 1782784 3222075 1990973 2831522...

array([  325,   520,  1013,  1257,  1663,  1686,  2178,  2289,  3324,
        3356,  4322,  5191,  5878,  7326,  7673,  8744,  8801,  9251,
        9785, 10137, 10305, 11540, 11551, 12193, 12216, 12539, 12987,
       14067, 15694, 15732, 15931, 16333, 17038], dtype=int32)

In [53]:
#the tracks are of artist 414349 415522 232974 82870 82870...
unique(onlyArtists)[targetPlaylistArtistReduced[1].nonzero()[1]]
#è un casino ma i dati sono corretti

array([ 12709,  18048,  28796,  35065,  44284,  44882,  55438,  57655,
        82870,  83519, 115017, 145129, 160964, 198856, 207330, 231313,
       232974, 245419, 260057, 269361, 273986, 308117, 308599, 325897,
       326285, 335250, 346606, 369258, 414349, 415522, 425607, 436838,
       456649], dtype=int32)

In [66]:
orderedTargetTracksIDs=load("targetTracksOrdered.npy")
orderedTargetTracksIDs

array([1563309, 1363985, 3705881, ..., 2005630, 4432895, 3626362], dtype=int32)

In [71]:
allTracksArtistReduced.getrow(0)

<1x17536 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [76]:
firstOrderedTrack=allTracksArtistReduced.getrow(where(allTrackIDs==orderedTracksIDs[0])[0][0])
for orderedTrackID in orderedTargetTracksIDs[1:]:
    trackToStack=allTracksArtistReduced.getrow(where(allTrackIDs==orderedTrackID)[0][0])
    firstOrderedTrack=vstack((firstOrderedTrack, trackToStack))

In [78]:
save_npz("targetTracksArtistReduced.npz",firstOrderedTrack)

In [85]:
#verifichiamo anche qui
targetTracksArtistReduced=load_npz("targetTracksArtistReduced.npz")
#la traccia in posizione 2 è la terza in classifica, con ID 3705881
targetTracksArtistReduced[2].nonzero()[1]#10764 è la posizione negli artisti filtrati
unique(onlyArtists)[10764]#286091 è l'id dell'artista in posizione 10764
#anche qui un casino ma funziona

286091